In [ ]:
'''
# dataset load
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="8CzHJJr0mbSZN5Yndqlp")
project = rf.workspace("project-986i8").project("drone-uskpc")
dataset = project.version(1).download("tensorflow")

'''

In [ ]:
'''
# dataset load
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="8CzHJJr0mbSZN5Yndqlp")
project = rf.workspace("project-986i8").project("drone-uskpc")
dataset = project.version(1).download("yolov8")

'''

### all data collection

In [ ]:
#-- Create directory structure as follows:
'''
done_ds:
    train:
        images
        labels

    valid:
        images
        labels

    test:
        images
        labels
'''
import os
import shutil


#-- Create Empty folders -------------------------------------------------------------

#-- Define the path for the main folder --
main_folder = 'drone_ds'

#-- Define subfolders --
subfolders = ['train', 'valid', 'test']
subsubfolders = ['images', 'labels']

#-- save created directories as a dict --
dir_dict = {}

#-- Create the main folder --
os.makedirs(main_folder, exist_ok=True)

#-- Create subfolders and sub-subfolders --
for folder in subfolders:
    os.makedirs(os.path.join(main_folder, folder), exist_ok=True)
   
    for subfolder in subsubfolders:
        new_dir = os.path.join(main_folder, folder, subfolder)
        os.makedirs(new_dir, exist_ok=True)
        dir_dict[(folder,subfolder)] = new_dir


# print(dir_dict)
#--------------------------------------------------------------------------------------

In [ ]:
#-- fild paths for images and labeles --------------------------------------------------
def find_paths(directory, extensions):
    
    file_paths = set() 
    
    #-- Walk through the directory tree --
    for root, _, files in os.walk(directory):        
        for file in files:            
            if any(file.endswith(ext) for ext in extensions):
                file_paths.add(root)                
                break
    
    return file_paths
#--------------------------------------------------------------------------------------

#-- find all paths for images and labels ----------------------------------------------

#-- Define the directory to search in --
directory_to_search = '/kaggle/input'

#-- Define the file extensions you want to search for --
image_extensions = {'.jpg', '.jpeg', '.png', '.JPEG'}  
text_extensions = {'.txt'}  

#-- Find paths to image files --
images_paths = find_paths(directory_to_search, image_extensions)

#-- Find paths to label files --
labels_paths = find_paths(directory_to_search, text_extensions)

#-- Print the paths --
print("Image paths:")
for img_path in images_paths:
    print(img_path)

print("\nText paths:")
for lbl_path in labels_paths:
    print(lbl_path)
#--------------------------------------------------------------------------------------

In [ ]:
#-- Copy all images and labels from input to drone_ds folder ----------------------------

tags = ['train', 'valid', 'test']

#-- copy images --
for img_path in images_paths:
    if 'train' in img_path or 'test' in img_path or 'valid' in img_path:
        if 'train' in img_path: 
            tag = 'train'

        elif 'valid' in img_path: 
            tag = 'valid'

        elif 'test' in img_path: 
            tag = 'test'
         
        
        #-- log --
        print(f'copy {tag} images from {img_path} ---- ')

        #-- set source and destination --
        source_dir = img_path
        dest_dir = dir_dict[(tag,'images')]

        #-- copy images --
        files = os.listdir(source_dir)            
        for file in files:
            source_file_path = os.path.join(source_dir, file)
            destination_file_path = os.path.join(dest_dir, file)
            shutil.copy(source_file_path, destination_file_path)



#-- copy labels --
for lbl_path in labels_paths:
    if 'train' in lbl_path or 'test' in lbl_path or 'valid' in lbl_path:
        if 'train' in lbl_path: 
            tag = 'train'

        elif 'valid' in lbl_path: 
            tag = 'valid'

        elif 'test' in lbl_path: 
            tag = 'test'
        
            
        #-- log --
        print(f'copy {tag} labels from {lbl_path} ---- ')

        #-- set source and destination --
        source_dir = lbl_path
        dest_dir = dir_dict[(tag,'labels')]

        #-- copy images --
        files = os.listdir(source_dir)            
        for file in files:
            source_file_path = os.path.join(source_dir, file)
            destination_file_path = os.path.join(dest_dir, file)
            shutil.copy(source_file_path, destination_file_path)
#--------------------------------------------------------------------------------------    

In [ ]:
import os 
import random
import shutil

random.seed(0)

tags = ['train', 'valid', 'test']
image_divide_path = []
label_divide_path = []

#         if 'train' not in img and 'test' not in img and 'valid' not in img:
#         if 'train' not in lbl and 'test' not in lbl and 'valid' not in lbl:

#-- copy images --
for img_path in images_paths:
    if 'train' not in img_path and 'test' not in img_path and 'valid' not in img_path:
        print(img_path)
        images = os.listdir(img_path)
        print(len(images))
        for img in images:
            image_path = os.path.join(img_path, img)
            image_divide_path.append(image_path)
        
for lbl_path in labels_paths:
    if 'train' not in lbl_path and 'test' not in lbl_path and 'valid' not in lbl_path:
        print(lbl_path)
        labels = os.listdir(lbl_path)
        print(len(labels))
        for lbl in labels:
            label_path = os.path.join(lbl_path, lbl)
            label_divide_path.append(label_path)
# Shuffle the list to randomize the order

random.shuffle(image_divide_path)

# Calculate the number of files for each group
total_files = len(image_divide_path)
train_count = int(total_files * 0.70)
valid_count = int(total_files * 0.20)
test_count = total_files - train_count - valid_count

image_label_map = {}
for image_path in image_divide_path:
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    for label_path in label_divide_path:
        label_name = os.path.splitext(os.path.basename(label_path))[0]
        if image_name == label_name:
            image_label_map[image_path] = label_path
            break
# print("dictionary made")
# Use the image_label_map to copy files to the appropriate folders
for i, image_path in enumerate(image_divide_path):
    if i < train_count:
        dest_dir = dir_dict[('train','images')]
        dest_dir_lbl = dir_dict[('train','labels')]
    elif i < train_count + valid_count:
        dest_dir = dir_dict[('valid','images')]
        dest_dir_lbl = dir_dict[('valid','labels')]
    else:
        dest_dir = dir_dict[('test','images')]
        dest_dir_lbl = dir_dict[('test','labels')]

    shutil.copy(image_path, dest_dir)
    shutil.copy(image_label_map[image_path], dest_dir_lbl)
# print("the directories made")

### end collection

In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re
# import pydicom
import warnings

from PIL import Image
import cv2

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.core.transforms_interface import ImageOnlyTransform

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt
import random
paddingSize= 0

warnings.filterwarnings("ignore")


# DIR_INPUT = '/kaggle/working/Drone-1'
DIR_INPUT = '/kaggle/working/drone_ds'
DIR_TRAIN = f'{DIR_INPUT}/train'
DIR_VALID = f'{DIR_INPUT}/valid'
DIR_TEST = f'{DIR_INPUT}/test'




In [ ]:
import os
def read_dataset(folder_path):
  fnames=os.listdir(f"{folder_path}/images/")
  df = pd.DataFrame()
  filename=[]
  class_num,xmin,ymin,xmax,ymax=[],[],[],[],[]
  for fname in fnames:
    if fname[-3:]== 'jpg'or fname[-3:]== 'JPG' or fname[-3:]== 'png' or fname[-3:]== 'PNG':
        with open(f"{folder_path}/labels/{fname[:-3]}txt") as f:
          data=f.readline()
          boxes= data.split()
          if len(boxes)==5:
#             if fname=='bird_44.jpg':
#                 print("here")
#             else:
                filename.append(fname)
                _, x_center, y_center, width, height = map(float,boxes)
            ##### check for 0 ymax and xmax
#             if (int((y_center + (height / 2))*640))<=0 or (int((x_center + (width / 2))*640))<=0:
#               print(data,'file:',fname)
            #####
                ymax.append(int((y_center + (height / 2))*640) +1)
                xmax.append(int((x_center + (width / 2))*640) +1)
                ymin.append(int((y_center - (height / 2))*640))
                xmin.append(int((x_center - (width / 2))*640))
            
                class_num.append(int(1))
            
          else:
#             print(f"{filename} dosent contain box data")
            
            filename.append(fname)
            ymax.append(int(640))
            xmax.append(int(640))
            ymin.append(int(0))
            xmin.append(int(0))
            class_num.append(int(0))
        
    elif fname[-4:]== 'jpeg' or fname[-4:]== 'JPEG':
        with open(f"{folder_path}/labels/{fname[:-4]}txt") as f:
          data=f.readline()
          boxes= data.split()
          if len(boxes)==5:
            filename.append(fname)
            _, x_center, y_center, width, height = map(float,boxes)
            ##### check for 0 ymax and xmax
#             if (int((y_center + (height / 2))*640))<=0 or (int((x_center + (width / 2))*640))<=0:
#               print(data,'file:',fname)
            ######
            ymax.append(int((y_center + (height / 2))*640) +1)
            xmax.append(int((x_center + (width / 2))*640) +1)
            ymin.append(int((y_center - (height / 2))*640))
            xmin.append(int((x_center - (width / 2))*640))
            
            class_num.append(int(1))

          else:
#             print(f"{filename} dosent contain box data")
            
            filename.append(fname)
            ymax.append(int(640))
            xmax.append(int(640))
            ymin.append(int(0))
            xmin.append(int(0))
            class_num.append(int(0))
    else :
#         continue
        print(fname[-5:])
##_______

##_______
        
        

  df['filename']=filename
  df['ymax']=ymax
  df['ymin']=ymin
  df['xmax']=xmax
  df['xmin']=xmin
####
  df['class_id']=class_num
    
  return df
# testing_df = read_dataset(DIR_TRAIN)

In [ ]:
# train_df = pd.read_csv(f'{DIR_TRAIN}/_annotations.csv') #[:100]
train_df = read_dataset(DIR_TRAIN) #[:30]
print("df Shape: "+str(train_df.shape))
# print("No Of Classes: "+str(train_df["class"].nunique()))
# train_df['class_id']=1
train_df.sort_values(by='filename').head(10)

In [ ]:
# train_df = pd.read_csv(f'{DIR_TRAIN}/_annotations.csv') #[:100]
# print("df Shape: "+str(train_df.shape))
# print("No Of Classes: "+str(train_df["class"].nunique()))
# train_df['class_id']=1
# train_df.sort_values(by='filename').head(10)

In [ ]:
# valid_df = pd.read_csv(f'{DIR_VALID}/_annotations.csv')#[:100]
valid_df = read_dataset(DIR_VALID) #[:10]
print("df Shape: "+str(valid_df.shape))
# print("No Of Classes: "+str(valid_df["class"].nunique()))
# valid_df['class_id']=1
valid_df.sort_values(by='filename').head(10)

In [ ]:
# Thanks -  https://www.kaggle.com/pestipeti/
class VinBigDataset(Dataset): #Class to load Training Data

    def __init__(self, dataframe, image_dir, transforms=None,stat = 'Train'):
        super().__init__()

        self.image_ids = dataframe["filename"].unique()
        self.df = dataframe
        self.image_dir = image_dir
        self.transforms = transforms
        self.stat = stat

    def __getitem__(self, index):
        if self.stat == 'Train':

            image_id = self.image_ids[index]
            records = self.df[(self.df['filename'] == image_id)]
            records = records.reset_index(drop=True)

            # dicom = pydicom.dcmread(f"{self.image_dir}/{image_id}.dicom")

            # image = dicom.pixel_array

            # if "PhotometricInterpretation" in dicom:
            #     if dicom.PhotometricInterpretation == "MONOCHROME1":
            #         image = np.amax(image) - image

            # intercept = dicom.RescaleIntercept if "RescaleIntercept" in dicom else 0.0
            # slope = dicom.RescaleSlope if "RescaleSlope" in dicom else 1.0

            # if slope != 1:
            #     image = slope * image.astype(np.float64)
            #     image = image.astype(np.int16)


            # image += np.int16(intercept)

            # image = np.stack([image, image, image])
            # image = image.astype('float32')
            # image = image - image.min()
            # image = image / image.max()
            # image = image * 255.0
            # image = image.transpose(1,2,0)
            image = cv2.imread(f"{self.image_dir}/images/{image_id}")

            try:         
                image = cv2.resize(image,(640,640))
                image = image / 255.0
            except Exception as e:
                width, height = 640, 640
                background_color = (0.0, 0.0, 0.0)  # White

                # Create a blank white image
                image = np.full((height, width, 3), background_color, dtype=np.uint8)
                print(f"{image_id}:{e}")#, image)
#                 continue
                
#             image = cv2.resize(image,(640,640))
#             image = image / 255.0

            # if records.loc[0, "class_id"] == 0:
            #     records = records.loc[[0], :]

            boxes = records[['xmin', 'ymin', 'xmax', 'ymax']].values
            area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
            area = torch.as_tensor(area, dtype=torch.float32)
            labels = torch.tensor(records["class_id"].values, dtype=torch.int64)

            # suppose all instances are not crowd
            iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)

            target = {}
            target['boxes'] = boxes
            target['labels'] = labels
            # target['image_id'] = torch.tensor([index])
            # target['area'] = area
            # target['iscrowd'] = iscrowd

            if self.transforms:
                sample = {
                    'image': image,
                    'bboxes': target['boxes'],
                    'labels': labels
                }
                sample = self.transforms(**sample)
                image = sample['image']

                target['boxes'] = torch.tensor(sample['bboxes'])

            if target["boxes"].shape[0] == 0:
                # Albumentation cuts the target (class 14, 1x1px in the corner)
                target["boxes"] = torch.from_numpy(np.array([[0.0, 0.0, 1.0, 1.0]]))
                # target["area"] = torch.tensor([1.0], dtype=torch.float32)
                target["labels"] = torch.tensor([0], dtype=torch.int64)

            return image, target, image_id

        else:

            image_id = self.image_ids[index]
            records = self.df[(self.df['filename'] == image_id)]
            records = records.reset_index(drop=True)

            # dicom = pydicom.dcmread(f"{self.image_dir}/{image_id}.dicom")

            # image = dicom.pixel_array

            # intercept = dicom.RescaleIntercept if "RescaleIntercept" in dicom else 0.0
            # slope = dicom.RescaleSlope if "RescaleSlope" in dicom else 1.0

            # if slope != 1:
            #     image = slope * image.astype(np.float64)
            #     image = image.astype(np.int16)

            # image += np.int16(intercept)

            # image = np.stack([image, image, image])
            # image = image.astype('float32')
            # image = image - image.min()
            # image = image / image.max()
            # image = image * 255.0
            # image = image.transpose(1,2,0)
            image = cv2.imread(f"{self.image_dir}/images/{image_id}")

            try:
                image = cv2.resize(image,(640,640))
                image = image / 255.0
            except Exception as e:
                width, height = 640, 640
                background_color = (0.0, 0.0, 0.0)  # White

                # Create a blank white image
                image = np.full((height, width, 3), background_color, dtype=np.uint8)
                print(f"{image_id}:{e}")#, image)
                

            if self.transforms:
                sample = {
                    'image': image,
                }
                sample = self.transforms(**sample)
                image = sample['image']

            return image, image_id

    def __len__(self):
        return self.image_ids.shape[0]

In [ ]:
# import torchvision
# help(torchvision.models.detection.FasterRCNN)

In [ ]:
# import torchvision
# dir(torchvision.models.detection)

In [ ]:
# pip install torchsummary


In [ ]:
import torchvision


# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)

# from torchinfo import summary
# summary(model)
# model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(pretrained = True)


# 'fasterrcnn_mobilenet_v3_large_320_fpn',
# 'fasterrcnn_mobilenet_v3_large_fpn',
# 'fasterrcnn_resnet50_fpn',
# 'fasterrcnn_resnet50_fpn_v2

In [ ]:
#### VGG16
# from torchvision.models.detection import FasterRCNN
# from torchvision.models.detection.rpn import AnchorGenerator
# backbone = torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT).features
# backbone.out_channels = 512
# anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),aspect_ratios=((0.5, 1.0, 2.0),))
# roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],output_size=7, sampling_ratio=2)
#  # put the pieces together inside a FasterRCNN model
# model = FasterRCNN(backbone,num_classes=2,rpn_anchor_generator=anchor_generator,box_roi_pool=roi_pooler)
# model

In [ ]:
num_classes = 2 # 

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor =  FastRCNNPredictor(in_features,num_classes)
# replace the pre-trained head with a new one
# model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = VinBigDataset(train_df, DIR_TRAIN, ToTensorV2(p=1.0))#, get_train_transform())
valid_dataset = VinBigDataset(valid_df, DIR_VALID, ToTensorV2(p=1.0))#, get_valid_transform())


# split the dataset in train and test set
indices = torch.randperm(len(train_dataset)).tolist()
# Create train and validate data loader
train_data_loader = DataLoader(
    train_dataset,
    batch_size= 8 , # 8
    shuffle=True,
    num_workers=0,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size= 8,# 8
    shuffle=False,
    num_workers=0,
    collate_fn=collate_fn
)

In [ ]:
del train_df
del valid_df
# del valid_data_loader

In [ ]:
# Train dataset sample
images, targets, image_ids = next(iter(train_data_loader))
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

for number in random.sample([1,2,3],3):
#   boxes = targets[number]['boxes'].cpu().numpy().astype(np.int32)
  boxes = (targets[number]['boxes'].cpu().numpy()).astype(np.int32)
  img = images[number].permute(1,2,0).cpu().numpy()
  labels= targets[number]['labels'].cpu().numpy().astype(np.int32)
  fig, ax = plt.subplots(1, 1, figsize=(16, 8))

  for i in range(len(boxes)):
      img = cv2.rectangle(img,(boxes[i][0]+paddingSize,boxes[i][1]+paddingSize),(boxes[i][2]+paddingSize,boxes[i][3]+paddingSize),(255,0,0),2)
      #print(le.inverse_transform([labels[i]-1])[0])
      #print(label_to_name(labels[i]), (int(boxes[i][0]), int(boxes[i][1])))
      # img = cv2.putText(img, labels[i], (int(boxes[i][0]), int(boxes[i][1])), cv2.FONT_HERSHEY_TRIPLEX,1, (255,0,0), 2, cv2.LINE_AA)

  ax.set_axis_off()
  ax.imshow(img)

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [ ]:
# Load Model

# model = torch.load('/kaggle/input/model-new2/mdl_new2.pth')#,map_location=torch.device('cpu'))


In [ ]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

num_epochs = 5  #Low epoch to save GPU time

In [ ]:
def voc_ap(rec, prec, use_07_metric=True):
    """ ap = voc_ap(rec, prec, [use_07_metric])
    Compute VOC AP given precision and recall.
    If use_07_metric is true, uses the
    VOC 07 11 point method (default:True).
    """
    if use_07_metric:
        # 11 point metric
        ap = 0.
        for t in np.arange(0., 1.1, 0.1):
            if np.sum(rec >= t) == 0:
                p = 0
            else:
                p = np.max(prec[rec >= t])
            ap = ap + p / 11.
    else:
        # correct AP calculation
        # first append sentinel values at the end
        mrec = np.concatenate(([0.], rec, [1.]))
        mpre = np.concatenate(([0.], prec, [0.]))

        # compute the precision envelope
        for i in range(mpre.size - 1, 0, -1):
            mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

        # to calculate area under PR curve, look for points
        # where X axis (recall) changes value
        i = np.where(mrec[1:] != mrec[:-1])[0]

        # and sum (\Delta recall) * prec
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap


In [ ]:
loss_hist = Averager()
itr = 1
lossHistoryiter = []
lossHistoryepoch = []

mAP_Historyepoch = []

ovthresh=0.5
use_07_metric=True
tp = 0
fp = 0
npos = 0
Total_score = 0
Total_overlaps = 0
best_mAP = 0

import time
start = time.time()
import tqdm
for epoch in range(num_epochs):
    loss_hist.reset()
    
    model.train()

    for images, targets, image_ids in tqdm.tqdm(train_data_loader):

        images = list(image.float().to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
#         print(type(loss_dict))
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()

        loss_hist.send(loss_value)
        lossHistoryiter.append(loss_value)
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        if itr % 50 == 0:
            print(f"Iteration #{itr} loss: {loss_value}",end="\r")
        itr += 1

    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()
    lossHistoryepoch.append(loss_hist.value)
    print(f"Epoch #{epoch} loss: {loss_hist.value}")
    
    ## Validation
    model.eval()
    cpu_device = torch.device("cpu")

    for images, targets, image_ids in tqdm.tqdm(valid_data_loader):
        images = list(image.float().to(device) for image in images)
    #     targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    #     outputs = model(images)

        with torch.no_grad():
            outputs = model(images)

        for tar,out in zip(targets,outputs):
    #         print(tar['boxes'].size(),out['boxes'].size())
            score = out['scores'].sum().cpu().detach().numpy()
    #         Total_score = Total_score + score

            bb = tar['boxes'].cpu().detach().numpy()
            BBGT = out['boxes'].cpu().detach().numpy()
            ixmin = np.maximum(BBGT[:, 0], bb[0,0])
            iymin = np.maximum(BBGT[:, 1], bb[0,1])
            ixmax = np.minimum(BBGT[:, 2], bb[0,2])
            iymax = np.minimum(BBGT[:, 3], bb[0,3])
            iw = np.maximum(ixmax - ixmin, 0.)
            ih = np.maximum(iymax - iymin, 0.)
            inters = iw * ih
            uni = ((bb[0,2] - bb[0,0]) * (bb[0,3] - bb[0,1]) +
                    (BBGT[:, 2] - BBGT[:, 0]) *
                    (BBGT[:, 3] - BBGT[:, 1]) - inters)
            overlaps = inters / uni
    #         print(overlaps)

            if overlaps.size==0:
                continue
 
            ovmax = np.max(overlaps)
            jmax = np.argmax(overlaps)
            if ovmax > ovthresh:
                tp+=1
            else:
                fp+=1
            npos+=bb.shape[0]
    rec = tp / float(npos)
    prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
    ap = voc_ap(rec, prec, use_07_metric)
    print(f"Epoch #{epoch} mAP: {ap}")
    mAP_Historyepoch.append(ap)
        
    # save best validation
    if best_mAP<ap:
        torch.save(model, '/kaggle/working/mobilenet_bm_p1.pth')
        best_mAP = ap

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Time taken to Train the model :{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))




In [ ]:
# Save Model

# torch.save(model, '/kaggle/working/mobilenet_v3_P1.pth')


In [ ]:
# Plot training loss
import matplotlib.pyplot as plt
# x = [i for i in range(num_epochs)]
y = lossHistoryepoch

# Plot scatter plot of training loss
x = np.arange(1, num_epochs+1)
colors = y  # Use loss values as colors

plt.plot(x, y, marker='o', linestyle='-')#, color=plt.cm.viridis(colors))
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
# plt.colorbar(label='Loss')
plt.grid(True)
plt.show()
# plt.savefig('plot.jpg')

In [ ]:
# Plot validation mAP
import matplotlib.pyplot as plt
# x = [i for i in range(num_epochs)]
z = mAP_Historyepoch

# Plot scatter plot of training loss
x = np.arange(1, num_epochs+1)
colors = y  # Use loss values as colors

plt.plot(x, z, marker='o', linestyle='-')#, color=plt.cm.viridis(colors))
plt.xlabel('Epoch')
plt.ylabel('mAP')
plt.title('Validation mAP')
# plt.colorbar(label='Loss')
plt.grid(True)
plt.show()
# plt.savefig('plot.jpg')

In [ ]:
DIR_TEST = f'{DIR_INPUT}/test'
# test_df = pd.read_csv(f'{DIR_TEST}/_annotations.csv') #[:10]
test_df = read_dataset(DIR_TEST) 
print("df Shape: "+str(test_df.shape))
# print("No Of Classes: "+str(test_df["class"].nunique()))
# test_df['class_id']=1
test_df.sort_values(by='filename').head(10)


In [ ]:
# labels =  targets[1]['labels'].cpu().numpy()
model.eval()
cpu_device = torch.device("cpu")

In [ ]:
test_dataset = VinBigDataset(test_df, DIR_TEST, ToTensorV2(p=1.0))#,"Test")

test_data_loader = DataLoader(
    test_dataset,
    batch_size= 8, # 8
    shuffle=False,
    num_workers=0,
    drop_last=False,
    collate_fn=collate_fn
)

In [ ]:
del test_df

In [ ]:
def format_prediction_string(labels, boxes, scores):
    pred_strings = []
    for j in zip(labels, scores, boxes):
        pred_strings.append("{0} {1:.4f} {2} {3} {4} {5}".format(
            j[0], j[1], j[2][0], j[2][1], j[2][2], j[2][3]))

    return " ".join(pred_strings)

In [ ]:
# Test dataset sample
images,target, image_ids = next(iter(test_data_loader))
images = list(image.to(device) for image in images)

for number in random.sample([1,2,3],3):
  img = images[number].permute(1,2,0).cpu().numpy()
  #labels= targets[number]['labels'].cpu().numpy().astype(np.int32)
  fig, ax = plt.subplots(1, 1, figsize=(16, 8))
  ax.set_axis_off()
  ax.imshow(img)

In [ ]:
### Test images
images,target, image_ids = next(iter(test_data_loader))
images = list(img.float().to(device) for img in images)

outputs = model(images)
outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]


boxes = outputs[0]['boxes'].cpu().detach().numpy().astype(np.int32)
img = images[0].permute(1,2,0).cpu().detach().numpy()
labels= outputs[0]['labels'].cpu().detach().numpy().astype(np.int32)
score = outputs[0]['scores']
# print(score)

fig, ax = plt.subplots(1,1,figsize=(16, 8))

img = cv2.cvtColor(np.float32(img), cv2.COLOR_RGB2BGR)
for i in range(len(boxes)):
  img = cv2.rectangle(img,(boxes[i][0]+paddingSize,boxes[i][1]+paddingSize),(boxes[i][2]+paddingSize,boxes[i][3]+paddingSize),(255,0,0),2)
  #print(le.inverse_transform([labels[i]-1])[0])
#   print(label_to_name(labels[i]), (boxes[i][0]+paddingSize,boxes[i][1]+paddingSize),(boxes[i][2]+paddingSize,boxes[i][3]+paddingSize))
  # img = cv2.putText(img, label_to_name(labels[i]), (int(boxes[i][0]), int(boxes[i][1])), cv2.FONT_HERSHEY_TRIPLEX,3, (255,0,0), 3, cv2.LINE_AA)
t_boxes = target[0]['boxes'].cpu().detach().numpy().astype(np.int32)
for i in range(len(t_boxes)):
  img = cv2.rectangle(img,(t_boxes[i][0]+paddingSize,t_boxes[i][1]+paddingSize),(t_boxes[i][2]+paddingSize,t_boxes[i][3]+paddingSize),(0,0,255),2)
  
ax.set_axis_off()
ax.imshow(img)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import time
import tqdm

ovthresh=0.5
use_07_metric=True
tp = 0
fp = 0
npos = 0
# def voc_ap(rec, prec, use_07_metric=True):
#     """ ap = voc_ap(rec, prec, [use_07_metric])
#     Compute VOC AP given precision and recall.
#     If use_07_metric is true, uses the
#     VOC 07 11 point method (default:True).
#     """
#     if use_07_metric:
#         # 11 point metric
#         ap = 0.
#         for t in np.arange(0., 1.1, 0.1):
#             if np.sum(rec >= t) == 0:
#                 p = 0
#             else:
#                 p = np.max(prec[rec >= t])
#             ap = ap + p / 11.
#     else:
#         # correct AP calculation
#         # first append sentinel values at the end
#         mrec = np.concatenate(([0.], rec, [1.]))
#         mpre = np.concatenate(([0.], prec, [0.]))

#         # compute the precision envelope
#         for i in range(mpre.size - 1, 0, -1):
#             mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

#         # to calculate area under PR curve, look for points
#         # where X axis (recall) changes value
#         i = np.where(mrec[1:] != mrec[:-1])[0]

#         # and sum (\Delta recall) * prec
#         ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
#     return ap



Total_score = 0
Total_overlaps = 0
start = time.time()
for images, targets, image_ids in tqdm.tqdm(test_data_loader):
    images = list(image.float().to(device) for image in images)
#     targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
#     outputs = model(images)

    with torch.no_grad():
        outputs = model(images)
        
    for tar,out in zip(targets,outputs):
#         print(tar['boxes'].size(),out['boxes'].size())
        score = out['scores'].sum().cpu().detach().numpy()
#         Total_score = Total_score + score
        
        bb = tar['boxes'].cpu().detach().numpy()
        BBGT = out['boxes'].cpu().detach().numpy()
        ixmin = np.maximum(BBGT[:, 0], bb[0,0])
        iymin = np.maximum(BBGT[:, 1], bb[0,1])
        ixmax = np.minimum(BBGT[:, 2], bb[0,2])
        iymax = np.minimum(BBGT[:, 3], bb[0,3])
        iw = np.maximum(ixmax - ixmin, 0.)
        ih = np.maximum(iymax - iymin, 0.)
        inters = iw * ih
        uni = ((bb[0,2] - bb[0,0]) * (bb[0,3] - bb[0,1]) +
                (BBGT[:, 2] - BBGT[:, 0]) *
                (BBGT[:, 3] - BBGT[:, 1]) - inters)
        overlaps = inters / uni
#         print(overlaps)
        Total_overlaps = Total_overlaps + overlaps.sum()
        
        if overlaps.size==0:
#             overlaps=[0]
            continue
#         else:
#             continue
        Total_score = Total_score + score
        ovmax = np.max(overlaps)
        jmax = np.argmax(overlaps)
        if ovmax > ovthresh:
            tp+=1
        else:
            fp+=1
        npos+=bb.shape[0]
        

end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Time taken to Test the model :{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

rec = tp / float(npos)
prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
ap = voc_ap(rec, prec, use_07_metric)
IOU = Total_overlaps/npos
confidence = Total_score/npos

In [ ]:
print('mean average precision:',ap)
print('confidence :',confidence)
print("IoU:", IOU)

In [ ]:
# torch.cuda.empty_cache()

In [ ]:

'''

### predict images

folder_path = '/kaggle/input/imagefortest'

# Get list of files in the folder
file_list = os.listdir(folder_path)

# Read each image in the folder
list_image = []
for file_name in file_list:
#     if file_name.endswith('.jpg') or file_name.endswith('.png'):
        image_path = os.path.join(folder_path, file_name)
#         print(image_path)
        image = cv2.imread(image_path)
        if image is not None:
            list_image.append(image)
# print(len(list_image))
counter = 1
print(len(list_image))
for images in tqdm.tqdm(list_image):
    image = torch.tensor(images).to(device)
    image = image/255.0
    image = image.unsqueeze(0)
    image = image.permute(0,3,1,2)
#     image = image.squeeze(0)
    
#     images.to(device)
    start = time.time()
#     outputs = model(image)
    
    with torch.no_grad():
        outputs = model(image)
    print(len(outputs[0]))
    outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
    end = time.time()
    
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    print('Number :',counter )
    print("Time taken to per image the model :{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

    boxes = outputs[0]['boxes'].cpu().detach().numpy().astype(np.int32)
    img = image.permute(0,2,3,1).cpu().detach().numpy()
    img = img.squeeze(0)
    labels= outputs[0]['labels'].cpu().detach().numpy().astype(np.int32)
    score = outputs[0]['scores'].cpu().detach().numpy()
    print('label:',labels ,'-  score:', score)
    

    fig, ax = plt.subplots(1, 1)#, figsize=(8, 5))
    font = cv2.FONT_HERSHEY_SIMPLEX
    img = cv2.cvtColor(np.float32(img), cv2.COLOR_RGB2BGR)
    for i in range(len(boxes)):
      if score[i]>0.20: 
        img = cv2.rectangle(img,(boxes[i][0]+paddingSize,boxes[i][1]+paddingSize),(boxes[i][2]+paddingSize,boxes[i][3]+paddingSize),(0,255,0),2)
        img = cv2.putText(img,str(score[i]),(boxes[i][0]+paddingSize,boxes[i][1]+paddingSize-10),font,0.5,(0,255,0),1)

    ax.set_axis_off()
    ax.imshow(img)
    fig.savefig(f'{counter}.jpeg',dpi = 600)
    
    counter = counter + 1
    
    
'''